In [0]:
from Helpers import *

sc = SparkContext.getOrCreate()

## Input Parameters

In [0]:
# Get Input Parameters
print(f"AWS_REGION: {AWS_REGION}")
print(f"LOAD_DATE: {LOAD_DATE}")
print(f"SOURCE_BUCKETS: {SOURCE_BUCKETS}")
print(f"SNS_LOW: {SNS_TOPIC}")
print(f"DMZ_CATALOG: {DMZ_CATALOG}")
print(f"RWE_USERS_PATH: {RWE_USERS_PATH}")

## Manifest Processing

#### Age Calculation

In [0]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

@udf
def get_age_from_dob(date_of_birth):
    if not date_of_birth:
        return None
    dob = datetime.strptime(date_of_birth, '%Y-%m-%d') # Create datetime object from date_of_birth
    dob = datetime(dob.year, month=1, day=1) # Reset to the first day of the month
    age_in_years = relativedelta(datetime.now(), dob).years # Calculate age from dob
    return age_in_years

#### Create Account-Details & Write

In [0]:
print("Start Manifest File Processing")

# Create empty user dataframe using user_schema from Helpers.py
complete_users_df = spark.createDataFrame([], schema=user_schema)

# Iterate over each bucket in the SOURCE_BUCKETS list and union the files together
for bucket in SOURCE_BUCKETS:
    read_path = f's3://{bucket}/{RWE_USERS_PATH}/'
    try:
        user_df = spark.read.csv(read_path,
                sep=",", 
                header="true",
                schema=user_schema
                )
    except Exception as e:
        print(e)
        SNSNotifier(AWS_REGION, SNS_TOPIC, LOAD_DATE).notify_error(str(e))
        raise SourceMissingException(f"Error reading {read_path}. Confirm the path exists and permissions are set correctly")
    complete_users_df = complete_users_df.union(user_df)

# Get age from DOB
user_age_df = complete_users_df.withColumn("age", get_age_from_dob(col("DOB")))

# Refer Software Design Description
# #    AgeRange  AgeRangeLower  AgeRangeUpper
# 1      < 6          0              5
# 2     6 < 13        6              12
# 3     13 < 18       13             17
# 4     18 < 26       18             25
# 5     26 < 50       26             49
# 6     50 < 65       50             64
# 7     65 < 201      65            200

user_range_df = user_age_df.withColumn("agerangelower", 
                                        when(col("age") < 6, 0) \
                                        .when( (col("age") >= 6) & (col("age") <= 12), 6) \
                                        .when( (col("age") >= 13) & (col("age") <= 17), 13) \
                                        .when( (col("age") >= 18) & (col("age") <= 25), 18) \
                                        .when( (col("age") >= 26) & (col("age") <= 49), 26) \
                                        .when( (col("age") >= 50) & (col("age") <= 64), 50) \
                                        .when( (col("age") >= 65) & (col("age") <= 200), 65)) \
                            .withColumn("agerangeupper", 
                                        when(col("age") < 6, 5) \
                                        .when( (col("age") >= 6) & (col("age") <= 12), 12) \
                                        .when( (col("age") >= 13) & (col("age") <= 17), 17) \
                                        .when( (col("age") >= 18) & (col("age") <= 25), 25) \
                                        .when( (col("age") >= 26) & (col("age") <= 49), 49) \
                                        .when( (col("age") >= 50) & (col("age") <= 64), 64) \
                                        .when( (col("age") >= 65) & (col("age") <= 200), 200))     

# Deidentify
user_range_deid_df = user_range_df.withColumn("accountid", deid(col("ID")))

# Select cols
account_details_df = user_range_deid_df.select(
    col("accountid"),
    col("Country").alias("devicenationality"),
    col("agerangelower"),
    col("agerangeupper")
)

# Repartition so that only one file is written out
account_details_final_df = account_details_df.repartition(1)

account_details_final_df.cache()
num_accounts = account_details_final_df.count()
print(f"Number of accounts: {num_accounts}")


# Write out the account-details file
try:
    account_details_final_df.write.mode('overwrite') \
        .option('nullValue', None) \
        .saveAsTable(f"{DMZ_CATALOG}.{FSL1_FSL2_SCHEMA}.account_details")
    print('Done Writing Account-Details to FSL1&2 Schema')

    account_details_final_df.write.mode('overwrite') \
        .option('nullValue', None) \
        .saveAsTable(f"{DMZ_CATALOG}.{FSL3_SCHEMA}.account_details")
    print('Done Writing Account-Details to FSL3 Schema')
except Exception as e:
    print(e)
    SNSNotifier(AWS_REGION, SNS_TOPIC, LOAD_DATE).notify_error(str(e))
    sys.exit(f"Error writing account details")

SNSNotifier(AWS_REGION, SNS_TOPIC, LOAD_DATE).notify_success(f'Manifest Job Complete (account-details)\nAccounts: {num_accounts}')